In [1]:
# Importing packages
import copy
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from scipy import signal
import keras
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import keras
import keras_tuner as kt
from keras.utils import to_categorical
from keras.layers import Dense, Convolution1D, Convolution2D, MaxPool1D, Flatten, Dropout, AveragePooling1D
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils import class_weight
from sklearn.utils import resample
import warnings
import wfdb
import os
warnings.filterwarnings('ignore')

In [2]:
### Extract data from files
mitdb_list = [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 200, 201, 202, 203, 205, 207, 208, 209, 210, 212, 213, 214, 215, 217, 219, 220, 221, 222, 223, 228, 230, 231, 232, 233, 234]
NUM_FEATURES = 17

v_dataset_data = np.empty((1,NUM_FEATURES), dtype=float)
v_dataset_categ = np.empty(1, dtype=str)

for element in mitdb_list:

    print("##########")
    print("Current record: " + str(element))

    # Load all samples
    record = wfdb.rdrecord('src/mitdb/' + str(element))
    annotation = wfdb.rdann('src/mitdb/'+str(element), 'atr')

    # Convert annotations to useful ones
    # Read symbols and substitute them
    # Classes:  'N': Normal beat
    #           'S': Supraventricular premature
    #           'V': Premature ventricular contraction
    #           'F': Fusion of ventricular and normal beat
    #           'Q': Unclassifiable
    symbol_dict = { 'N':'N',
                    'S':'S', 'A':'S', 'a':'S', 'J':'S', 'e':'S', 'j':'S', 'x':'S',
                    'V':'V', 'L':'V', 'R':'V', 'E':'V', '[':'V', '!':'V', ']':'V',
                    'F':'F', 'f':'F',
                    'Q':'Q', '/':'Q', '|':'Q'}
    raw_ann_symbol = annotation.symbol.copy()
    raw_ann_sample = annotation.sample.copy()
    #raw_ann_symbol = [symbol_dict[key] if key in symbol_dict else key for key in raw_ann_symbol]
    raw_ann_symbol = [symbol_dict[key] if key in symbol_dict else 'Z' for key in raw_ann_symbol]

    # Delete non beat annotations
    if 'Z' in raw_ann_symbol:
        i_del = [i for i, x in enumerate(raw_ann_symbol) if x == 'Z']
        raw_ann_sample = np.delete(raw_ann_sample, i_del)
        raw_ann_symbol = np.delete(raw_ann_symbol, i_del)

    
    #del raw_ann_symbol[0]
    print(raw_ann_symbol)
    print(raw_ann_sample)
    print(len(raw_ann_symbol))
    print(len(raw_ann_sample))

    # Sampling frequency
    #FREQ_SAMPLE = 360
    #FREQ_SAMPLE = 1

    # Extract ECG data array
    v_data = record.p_signal
    v_data = v_data[:,0]

    # Define time array (Fs:360Hz)
    v_time = np.arange(0, v_data.size, 1, dtype=int)

    # Zeros array for x-axis
    axis_x = np.zeros_like(v_data)

##########
Current record: 100
['N' 'N' 'N' ... 'N' 'N' 'N']
[    77    370    662 ... 649484 649734 649991]
2273
2273
##########
Current record: 101
['N' 'N' 'N' ... 'N' 'N' 'N']
[    83    396    711 ... 649004 649372 649751]
1869
1869
##########
Current record: 102
['Q' 'Q' 'Q' ... 'Q' 'Q' 'Q']
[   136    410    697 ... 649244 649553 649852]
2187
2187
##########
Current record: 103
['N' 'N' 'N' ... 'N' 'N' 'N']
[   265    575    876 ... 649195 649534 649875]
2084
2084
##########
Current record: 104
['Q' 'Q' 'Q' ... 'Q' 'Q' 'Q']
[    73    316    615 ... 649300 649580 649876]
2229
2229
##########
Current record: 105
['N' 'N' 'N' ... 'N' 'N' 'N']
[   197    459    708 ... 649221 649471 649740]
2602
2602
##########
Current record: 106
['N' 'N' 'N' ... 'V' 'V' 'N']
[   351    724   1086 ... 649162 649340 649791]
2027
2027
##########
Current record: 107
['Q' 'Q' 'Q' ... 'Q' 'Q' 'Q']
[    95    272    569 ... 649129 649441 649748]
2137
2137
##########
Current record: 108
['N' 'N' 'N' ... 

In [4]:
def compress_signal(ecg_signal, threshold=100, interpolation_factor=4):
    peak_values = []
    peak_timestamps = []
    compressed_signal = []

    for i, value in enumerate(ecg_signal):
        if value > threshold:
            peak_values.append(value)
            peak_timestamps.append(i)

    for i in range(1, len(peak_values)):
        prev_peak_value = peak_values[i-1]
        prev_peak_timestamp = peak_timestamps[i-1]
        peak_value = peak_values[i]
        peak_timestamp = peak_timestamps[i]

        interpolation_step = (peak_timestamp - prev_peak_timestamp) / interpolation_factor
        for j in range(interpolation_factor):
            interpolated_value = prev_peak_value + (peak_value - prev_peak_value) * j / interpolation_factor
            compressed_signal.append(interpolated_value)

    return compressed_signal

# Example usage:
ecg_signal = np.random.randint(0, 200, size=256)  # Replace with your actual ECG signal
#ecg_signal = v_data[0]
compressed_signal = compress_signal(ecg_signal)
print(compressed_signal)


[153.0, 143.75, 134.5, 125.25, 116.0, 132.0, 148.0, 164.0, 180.0, 160.25, 140.5, 120.75, 101.0, 120.25, 139.5, 158.75, 178.0, 159.25, 140.5, 121.75, 103.0, 123.75, 144.5, 165.25, 186.0, 186.0, 186.0, 186.0, 186.0, 180.0, 174.0, 168.0, 162.0, 160.5, 159.0, 157.5, 156.0, 166.0, 176.0, 186.0, 196.0, 192.75, 189.5, 186.25, 183.0, 179.75, 176.5, 173.25, 170.0, 174.25, 178.5, 182.75, 187.0, 189.75, 192.5, 195.25, 198.0, 176.75, 155.5, 134.25, 113.0, 130.5, 148.0, 165.5, 183.0, 170.5, 158.0, 145.5, 133.0, 126.0, 119.0, 112.0, 105.0, 111.5, 118.0, 124.5, 131.0, 143.0, 155.0, 167.0, 179.0, 171.25, 163.5, 155.75, 148.0, 154.0, 160.0, 166.0, 172.0, 169.25, 166.5, 163.75, 161.0, 150.25, 139.5, 128.75, 118.0, 133.0, 148.0, 163.0, 178.0, 176.25, 174.5, 172.75, 171.0, 163.25, 155.5, 147.75, 140.0, 130.25, 120.5, 110.75, 101.0, 114.5, 128.0, 141.5, 155.0, 156.0, 157.0, 158.0, 159.0, 157.25, 155.5, 153.75, 152.0, 148.0, 144.0, 140.0, 136.0, 139.5, 143.0, 146.5, 150.0, 147.75, 145.5, 143.25, 141.0, 133.